In [ ]:
#pip install llama-index

In [ ]:
#pip install psycopg2-binary 

In [2]:
# Needs older version to work
#pip install langchain==0.0.188

In [34]:
import os
os.environ["OPENAI_API_KEY"] = ''

from llama_index import VectorStoreIndex, SimpleDirectoryReader, download_loader
from pathlib import Path
import streamlit as st # import the Streamlit library
from langchain.chains import LLMChain, SimpleSequentialChain # import LangChain libraries
from langchain.llms import OpenAI # import OpenAI model
from langchain.prompts import PromptTemplate # import PromptTemplate

# Single Loader

In [ ]:
# Source for different data loader connections
# https://llamahub.ai/

In [40]:
PandasCSVReader = download_loader("PandasCSVReader")

loader = PandasCSVReader()
documents = loader.load_data(file=Path('./bank.csv'))
index = VectorStoreIndex.from_documents(documents)

In [18]:
#query_engine = index.as_query_engine()
#query_engine.query("What is the customerid of Hargrave?")

# LangChain

In [41]:
from langchain.agents import Tool
tools = [
    Tool(
        name = "GPT Index",
        func=lambda q: str(index.as_query_engine().query(q)),
        description="The customer data set of a bank",
        return_direct=True
    ),
]

In [42]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.agents import initialize_agent

agent_type = "conversational-react-description"
memory = ConversationBufferMemory(memory_key="chat_history")
llm = OpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent=agent_type, memory =memory, verbose=True)

In [30]:
agent_chain.run(input="What is the average balance in my customer data set")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: GPT Index
Action Input: Average balance in customer data set
Observation: 
The average balance in the customer data set is approximately $112,945.


> Finished chain.


'\nThe average balance in the customer data set is approximately $112,945.'

In [31]:
agent_chain.run(input="Please give the detail behind your last awnser")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? No
AI: The average balance was calculated by taking the total balance of all customers in the data set and dividing it by the total number of customers. This gives us an average balance of $112,945.

> Finished chain.


'The average balance was calculated by taking the total balance of all customers in the data set and dividing it by the total number of customers. This gives us an average balance of $112,945.'

In [33]:
agent_chain.run(input="what is the count of n in my data set?")



> Entering new AgentExecutor chain...

Thought: Do I need to use a tool? Yes
Action: GPT Index
Action Input: customer data set
Observation: 
The customer data set includes information about 9833 customers, including their name, age, country, gender, number of years in current job, annual income, number of dependents, credit score, number of active credit cards, number of open credit accounts, total balance, and whether they have a personal loan.


> Finished chain.


'\nThe customer data set includes information about 9833 customers, including their name, age, country, gender, number of years in current job, annual income, number of dependents, credit score, number of active credit cards, number of open credit accounts, total balance, and whether they have a personal loan.'